In [1]:
from process_videos import process_video_with_flightaware
from pathlib import Path
from typing import List
import pandas as pd

/home/gabo/Personal/Universidad/04 - Cuarto Año/1er Semestre/Machine Learning/Proyectos/ml-project/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Checking connectivity to the model hosters, this may take a while. To bypass this check, set `DISABLE_MODEL_SOURCE_CHECK` to `True`.
/home/gabo/Personal/Universidad/04 - Cuarto Año/1er Semestre/Machine Learning/Proyectos/ml-project/.venv/lib/python3.11/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, ple

In [2]:

def find_video_files(directory: str, recursive: bool = True) -> List[str]:
    """
    Busca archivos de video en un directorio de forma recursiva.
    
    Args:
        directory: Ruta del directorio donde buscar los videos
        recursive: Si es True, busca en subdirectorios de forma recursiva
        
    Returns:
        Lista con las rutas completas de los archivos de video encontrados
    """
    # Extensiones de video más comunes
    video_extensions = [
        '.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv', '.mpeg', '.mpg',
        '.m4v', '.webm', '.3gp', '.3g2', '.m2ts', '.mts', '.ts', '.m2v',
        '.mxf', '.ogv', '.vob', '.asf', '.rm', '.rmvb', '.f4v', '.m4p',
        '.m4v', '.svi', '.m4b', '.m4r', '.3gpp', '.3g2', '.mxf'
    ]
    
    video_files = []
    path = Path(directory)
    
    if recursive:
        # Búsqueda recursiva
        for ext in video_extensions:
            video_files.extend([str(p) for p in path.rglob(f'*{ext}')])
            # Incluir también versiones en mayúsculas
            video_files.extend([str(p) for p in path.rglob(f'*{ext.upper()}')])
    else:
        # Búsqueda solo en el directorio actual
        for ext in video_extensions:
            video_files.extend([str(p) for p in path.glob(f'*{ext}')])
            video_files.extend([str(p) for p in path.glob(f'*{ext.upper()}')])
    
    return sorted(list(set(video_files)))  # Eliminar duplicados y ordenar


In [5]:
videos_path = find_video_files("/home/gabo/Personal/Work/Lita/Proyectos/VideoSearch/vid-chronicle-main/data/videos/TWR-HAV")
df = pd.DataFrame()

for vp in videos_path:
    if "+0,0" not in vp:
        continue

    result = process_video_with_flightaware(vp, frames_per_second_to_process=360)
    if not result.empty:
        df = pd.concat([df, result], ignore_index=True)

df = df.sort_values(by=["date", "time", "codigo_vuelo"])
df.to_csv("flightaware_results.csv", index=False)
print(f"Guardados {len(df)} registros en flightaware_results.csv")

🎬 Procesando video: /home/gabo/Personal/Work/Lita/Proyectos/VideoSearch/vid-chronicle-main/data/videos/TWR-HAV/2025/05/12/+0,0/2025-05-12T16:29:08Z_2025-05-12T16:44:08Z.mkv
FPS del video: 15.00
Se procesará aproximadamente cada 5400 frames para obtener 360 frame(s) por segundo.
Frame 5400 (Timestamp: 359.94s): [359.94s] System View Lists Alerts Msgs Help 16:35:09 NUEVAS-EXE CSQ503 2 M TREV D 150150 MKJS SF34/M N246 D n AAL1519 250250 MUCU a OFFR JBU1793 2 330330 A320/M N463 MROC D 7212 AVA009 1M 370370 SKBO . . AVA255 2 M 370370 - A20N/M N469 SKBO o n AVA036 2 A 360360 CMP484 2 C ABX22451M A320/M N447 KFLL 361 360 AVR D 350350 MPTO 370 B3SM/M N464 ASP AHDG SKBO . MCFL SWG6147 2 A CMP357 2 M 138↑ 270 fm1700 370370 o B38M/M N371 B738/M N455 a CYUL MPTO.78 DAL980 2 A . 360360 . KATL B752/M N445 CXE700 2 C 221 220 B190/M N264 o MKJP . NKS723 2 M CMP471 2 M 371370 350350 A321/M N480 . B39M/M N467 MKJP MPTO . AAL2217 D 350 CMP312 d 370 . FDX205 MKJK DAL1997 MKJKM 371370 3051210 SKBO MKJS B A

KeyboardInterrupt: 